<a href="https://colab.research.google.com/github/jsb58p/CS-441---Programming-assignment-1/blob/main/RR%2CQ%3D2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [420]:
# Shortest Remaining Time (SRT) algorithm

In [421]:
# Process class to represent a single process
class Process:
  def __init__(self, pid, arrival_time, burst_time, priority):
    self.pid = pid # unique process id
    self.arrival_time = arrival_time # time at which the process arrives
    self.burst_time = burst_time # total cpu time required by the process
    self.priority = priority # priority of the process
    self.completion_time = 0
    self.turnaround_time = 0
    self.waiting_time = 0
    self.remaining_time = burst_time

In [422]:
# Implement the SRT function
def shortest_remaining_time(processes_data):

  # Convert the input data into Process objects
  # Sort them by arrival time
  processes = [Process(*p) for p in processes_data]
  processes.sort(key=lambda x: x.arrival_time)

  # Initialize scheduling variables
  current_time = 0 # Track current time in simulation
  completed_processes = [] # Store processes that have finished running
  ready_queue = [] # Store processes that have arrived but not completed
  gantt_chart = [] # Store the execution timeline
  ready_index = 0
  Q = 2

  # check and add any newly arrived process to the ready queue
  def update_ready_queue():
    for process in processes:
      if(process.arrival_time <= current_time and
         process not in ready_queue and
         process not in completed_processes):
        ready_queue.append(process)

  # Find out when the next process will arrive
  def get_next_arrival_time():
    # get all processes that are in the ready queue
    # who's arrival time is in the future
    future_arrivals = [p.arrival_time for p in processes
                       if p.arrival_time > current_time and
                       p not in completed_processes]

    # return the earliest arrival time, or infinity if no more arrivals
    return min(future_arrivals) if future_arrivals else float('inf')


  # Main scheduling loop - continue until all processes are completed
  while len(completed_processes) < len(processes):

    # Step 1: Update the ready queue with any newly arrived process
    update_ready_queue()

    # Step 2: Handle case when no processes are ready to execute
    if not ready_queue:
      # Find the next process arrival time
      next_arrival = get_next_arrival_time()
      if current_time < next_arrival:
        # System will be idle
        gantt_chart.append(['IDLE'])
        current_time = next_arrival
      continue

    # Step 3: Select the process with the shortest remaining time
    # If tie, we will use arrival time as the arbitration rule
    if len(ready_queue) == 1:
      ready_index = 0
    current_process = ready_queue[ready_index]
    if current_process.remaining_time > Q:
      ready_index = (ready_index + 1) % len(ready_queue)




    # Step 4: Calculate the time slice
    next_arrival = get_next_arrival_time()
    time_slice = min(
        current_process.remaining_time,
        Q
    )

    # Step 5: Execute processes for the calculated time slice
    start_time = current_time # record the start time
    current_time += time_slice # advance the system clock
    current_process.remaining_time -= time_slice # update remaining time

    # update Gantt chart
    if not gantt_chart or gantt_chart[-1][0] != current_process.pid:
      # start a new entry if different process or first entry
      gantt_chart.append([current_process.pid, start_time, current_time])
    else:
      # update end time of current entry if same process
      gantt_chart[-1][2] = current_time

    # Step 6: Check if process has completed
    if current_process.remaining_time == 0:
      # update completion metrics
      current_process.completion_time = current_time

      # Turnaround time = completion time - arrival
      current_process.turnaround_time = (current_process.completion_time -
                                         current_process.arrival_time)

      # waiting time
      current_process.waiting_time = (current_process.turnaround_time -
                                      current_process.burst_time)

      # terminate the process
      # move it from the ready queue to the completed queue
      completed_processes.append(current_process)
      ready_queue.remove(current_process)

  return completed_processes, gantt_chart

In [423]:
def print_results(completed_processes, gantt_chart):
    """Print formatted scheduling results and metrics."""
    # Print Gantt chart showing execution order
    print("\nProcess Execution Order (Gantt Chart):")
    print("-" * 50)
    for entry in gantt_chart:
        if entry[0] == "IDLE":
            # Show idle time periods
            print(f"IDLE: {entry[1]} -> {entry[2]}")
        else:
            # Show process execution periods
            print(f"P{entry[0]}: {entry[1]} -> {entry[2]}")

    # Print detailed process metrics
    print("\nProcess Scheduling Details:")
    print("-" * 65)
    print("PID  Arrival  Burst  Completion  Turnaround  Waiting")
    print("-" * 65)

    # Print metrics for each process, sorted by PID
    for p in sorted(completed_processes, key=lambda x: x.pid):
        print(f"{p.pid:<5}{p.arrival_time:<9}{p.burst_time:<7}"
              f"{p.completion_time:<12}{p.turnaround_time:<12}"
              f"{p.waiting_time}")

    # Calculate and print average metrics
    avg_turnaround = sum(p.turnaround_time for p in completed_processes) / len(completed_processes)
    avg_waiting = sum(p.waiting_time for p in completed_processes) / len(completed_processes)

    print("-" * 65)
    print(f"Average Turnaround Time: {avg_turnaround:.2f}")
    print(f"Average Waiting Time: {avg_waiting:.2f}")

In [424]:
# process_id, arrival_time, burst_time, priority
processes = [[1, 0, 2, 2],
             [2, 1, 1, 1],
             [3, 2, 8, 4],
             [4, 3, 4, 2],
             [5, 4, 5, 3]]

completed_processes, gantt_chart = shortest_remaining_time(processes)
# print(completed_processes# )
print_results(completed_processes, gantt_chart)

list empty
ready index: 0
ready index: 0
ready index: 0
ready index: 1
ready index: 2
ready index: 0
ready index: 1
ready index: 1
ready index: 0
ready index: 1
list empty
ready index: 0

Process Execution Order (Gantt Chart):
--------------------------------------------------
P1: 0 -> 2
P2: 2 -> 3
P3: 3 -> 5
P4: 5 -> 7
P5: 7 -> 9
P3: 9 -> 11
P4: 11 -> 13
P5: 13 -> 15
P3: 15 -> 17
P5: 17 -> 18
P3: 18 -> 20

Process Scheduling Details:
-----------------------------------------------------------------
PID  Arrival  Burst  Completion  Turnaround  Waiting
-----------------------------------------------------------------
1    0        2      2           2           0
2    1        1      3           2           1
3    2        8      20          18          10
4    3        4      13          10          6
5    4        5      18          14          9
-----------------------------------------------------------------
Average Turnaround Time: 9.20
Average Waiting Time: 5.20
